In [ ]:
# Full Code for Automated Customer Feedback Analysis Research
# Hybrid Approach: Rule-Based + Machine Learning (Logistic Regression with TF-IDF)
# Focus: Detect Negative Product Feedback Impacts
# Environment: Google Colab
# Dataset: Loaded from Google Drive (dummy_customer_feedback.csv or replace with real dataset)
# Columns Used: 'review_text' (input), 'sentiment' (label: Positive, Negative, Neutral)
# Comparisons: Unigram, Bigram, Trigram features; NB, LR classifiers; Hybrid adjustments
# Outputs: Confusion Matrices, Evaluation Measures (P, R, F), Bar Charts, Accuracy vs. Sample Size

# Step 1: Install Dependencies
!pip install nltk scikit-learn matplotlib seaborn pandas numpy

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Step 2: Mount Google Drive and Load Dataset
from google.colab import drive
drive.mount('/content/drive')

# Update file_path to your dataset location in Google Drive
file_path = '/content/drive/MyDrive/ResearchData/dummy_customer_feedback.csv'  # Replace with your path
df = pd.read_csv(file_path)

# Step 3: Select Relevant Columns and Preprocess
# Select columns: 'review_text' for input, 'sentiment' for labels (remove IDs, etc.)
df_selected = df[['review_text', 'sentiment']]

# Clean text: Lowercase, remove punctuation
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

df_selected['review_text'] = df_selected['review_text'].apply(clean_text)

# Encode labels: Positive=2, Neutral=1, Negative=0
label_encoder = LabelEncoder()
df_selected['sentiment_encoded'] = label_encoder.fit_transform(df_selected['sentiment'])

# Check distribution
print(df_selected['sentiment'].value_counts())

# Step 4: Rule-Based Component
# Simple rule-based sentiment: Detect negations and negative keywords for hybrid adjustment
negative_keywords = ['bad', 'poor', 'terrible', 'disappointing', 'frustrating', 'broke', 'crash', 'slow', 'overheat', 'drain', 'weak', 'not good', 'not worth']
negation_words = ['not', 'no', 'never', 'none', 'nothing']

def rule_based_sentiment(text):
    tokens = word_tokenize(text)
    score = 0  # Positive: >0, Negative: <0, Neutral: 0
    negation = False
    for token in tokens:
        if token in negation_words:
            negation = True
            continue
        if token in negative_keywords:
            score -= 2 if negation else -1
        negation = False  # Reset after word
    if score < 0:
        return 0  # Negative
    elif score > 0:
        return 2  # Positive
    else:
        return 1  # Neutral

# Apply rule-based as a feature or for hybrid
df_selected['rule_sentiment'] = df_selected['review_text'].apply(rule_based_sentiment)

# Step 5: ML Component and Hybrid Model
# Function to train and evaluate models with different n-grams
def evaluate_model(ngram_range, classifier, model_name, sample_sizes=[10000, 15000, 25000]):
    results = {}
    accuracies = []
    for size in sample_sizes:
        if size > len(df_selected):
            size = len(df_selected)  # Adjust if dataset is small
        df_sample = df_selected.sample(n=size, random_state=42)
        
        X = df_sample['review_text']
        y = df_sample['sentiment_encoded']
        
        # TF-IDF Vectorizer
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=5000)
        X_vec = vectorizer.fit_transform(X)
        
        # Train-Test Split
        X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)
        
        # Train Classifier
        classifier.fit(X_train, y_train)
        y_pred_ml = classifier.predict(X_test)
        
        # Hybrid: Adjust ML predictions with rule-based (e.g., if rule detects negative impact, override to negative)
        y_pred_hybrid = []
        for idx, pred in enumerate(y_pred_ml):
            text = df_sample['review_text'].iloc[idx + len(df_sample) - len(y_test)]  # Align index
            rule_pred = rule_based_sentiment(text)
            if rule_pred == 0:  # If rule detects negative, override
                y_pred_hybrid.append(0)
            else:
                y_pred_hybrid.append(pred)
        
        # Evaluation
        cm = confusion_matrix(y_test, y_pred_hybrid)
        report = classification_report(y_test, y_pred_hybrid, output_dict=True, target_names=label_encoder.classes_)
        acc = accuracy_score(y_test, y_pred_hybrid)
        accuracies.append(acc)
        
        # Store for n-gram
        results[f'{model_name} - Size {size}'] = {'cm': cm, 'report': report, 'acc': acc}
    
    return results, accuracies, sample_sizes

# Models
classifiers = {
    'NB': MultinomialNB(),
    'LR': LogisticRegression(max_iter=1000)
}

# N-gram ranges
ngrams = {
    'Unigram': (1,1),
    'Bigram': (2,2),
    'Trigram': (3,3)
}

# Run evaluations
all_results = {}
all_accuracies = {'NB': [], 'LR': []}
sample_sizes_adjusted = [1000, 800, 600]  # Adjust for small dummy dataset; change for larger

for ngram_name, ngram_range in ngrams.items():
    for model_name, clf in classifiers.items():
        results, accs, _ = evaluate_model(ngram_range, clf, f'{ngram_name}-{model_name}', sample_sizes=sample_sizes_adjusted)
        all_results.update(results)
        if model_name == 'NB':
            all_accuracies['NB'].extend(accs)
        else:
            all_accuracies['LR'].extend(accs)

# Step 6: Generate Outputs like Images
# 6.1: Confusion Matrices and Evaluation Measures Tables
for key, res in all_results.items():
    print(f"\nN-gram Feature: {key}")
    cm = res['cm']
    report = res['report']
    
    # Confusion Matrix (assuming binary for simplicity; adjust for multi-class)
    # Note: Images show binary (Positive/Negative), but dataset has 3 classes. Simplify to binary for match.
    # Map Neutral to Positive for binary approximation
    print("Confusion Matrix:")
    print(pd.DataFrame(cm, index=label_encoder.classes_, columns=label_encoder.classes_))
    
    # Evaluation Measures Table
    measures = {
        'P': [report[label]['precision']*100 for label in label_encoder.classes_],
        'R': [report[label]['recall']*100 for label in label_encoder.classes_],
        'F': [report[label]['f1-score']*100 for label in label_encoder.classes_]
    }
    print("Evaluation Measures:")
    print(pd.DataFrame(measures, index=label_encoder.classes_))

# 6.2: Bar Chart for Evaluation Measures (Average across classes)
avg_measures = {
    'Precision': np.mean([report['weighted avg']['precision']*100 for res in all_results.values()]),
    'Recall': np.mean([report['weighted avg']['recall']*100 for res in all_results.values()]),
    'F-Measure': np.mean([report['weighted avg']['f1-score']*100 for res in all_results.values()])
}
fig, ax = plt.subplots()
ax.bar(avg_measures.keys(), avg_measures.values(), color=['blue', 'orange', 'gray'])
ax.set_title('Evaluation Measures')
ax.set_ylabel('Percentage (%)')
plt.show()

# 6.3: Classification Accuracy Bar Chart vs. No. of Reviews
fig, ax = plt.subplots()
width = 0.25
x = np.arange(len(sample_sizes_adjusted))
ax.bar(x - width/2, all_accuracies['NB'], width, label='NB', color='orange')
ax.bar(x + width/2, all_accuracies['LR'], width, label='LR', color='gray')
ax.set_xticks(x)
ax.set_xticklabels(sample_sizes_adjusted)
ax.set_title('Classification Accuracy')
ax.set_xlabel('No. of Reviews')
ax.set_ylabel('Accuracy (%)')
ax.legend()
plt.show()